# POC (e): compare two areas

## Copy request_api and mask function

In [ ]:
import cdsapi
import time

# a wrapper function to request ERA5 API
# f_name: the name of the file to be requested
# shape: the ARBITRARY shape of the area to be requested
#        if None, the whole world will be queried
#        otherwise, the request ERA5 API with the bounding box of the shape
# return: the path of the file downloaded
def request_era5_api(f_name, shape=None, day = ['05'], month = ['01']):
    fpath = f'data/download/{f_name}'
    if shape is None:
        area = [90, -180, -90, 180]
    else:
        west, sorth, east, north = shape.bounds
        area = [north, west, sorth, east]

    print('\n### ~~~~~~ ###')
    print('START requesting ERA5 API')
    start = time.time()

    c = cdsapi.Client()
    c.retrieve('reanalysis-era5-single-levels', {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': '2m_temperature',
        'year': '2023',
        'month': month,
        'day': day,
        'time': [
            '00:00',
            '01:00',
            '02:00',
            '03:00',
            '04:00',
            '05:00',
            '06:00',
            '07:00',
            '08:00',
            '09:00',
            '10:00',
            '11:00',
            '12:00',
            '13:00',
            '14:00',
            '15:00',
            '16:00',
            '17:00',
            '18:00',
            '19:00',
            '20:00',
            '21:00',
            '22:00',
            '23:00',
        ],
        'area': area,
    }, fpath)

    end = time.time()
    print(f'DONE requesting ERA5 API in {end - start} seconds')
    print('### ~~~~~~ ###\n')

    return fpath

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import folium
import time

# a function to mask the raster data with arbitrary shape
# fpath: the path of the raster data
# shape: the ARBITRARY shape
# return: the mask of the raster data, which is a boolean 2-D array
def shape_mask(fpath, shape):
    print('\n### ~~~~~~ ###')
    print('START masking raster data with arbitrary shape')
    start = time.time()

    gdf_shape = gpd.GeoDataFrame(geometry=[shape], crs=4326)

    ds = xr.open_dataset(fpath)
    print(f'shape of the whole raster data: {ds["t2m"].shape}')
    ds_2d = ds.isel(time=0)  # using a 2-D slice of the raster to construct the geospatail content of the pixels
    print(f'shape of the sliced raster data: {ds_2d["t2m"].shape}')

    # take a record of the lat/lon location in the raster data
    df_lat = pd.DataFrame(enumerate(ds_2d['latitude'].values), columns=['lat_index', 'latitude'])
    df_lon = pd.DataFrame(enumerate(ds_2d['longitude'].values), columns=['lon_index', 'longitude'])

    df_2d = ds_2d.to_dataframe()
    df_2d = df_2d.reset_index()
    df_2d = df_2d[['latitude', 'longitude', 't2m']]
    df_2d = df_2d.merge(df_lat, on='latitude')
    df_2d = df_2d.merge(df_lon, on='longitude')
    gdf_2d = gpd.GeoDataFrame(
        df_2d,
        geometry=gpd.points_from_xy(df_2d.longitude, df_2d.latitude),
        crs=4326,
    )  # construct the geospatail dataframe of the pixels
    gdf_masked = gdf_2d.sjoin(gdf_shape, how='inner', predicate="within")  # join the pixels with the shape, use GeoPandas spatial join

    # construct the mask of based on the lat/lon location of the pixels within the shape
    lat_index = gdf_masked['lat_index'].values
    lon_index = gdf_masked['lon_index'].values
    mask = np.zeros(ds_2d['t2m'].shape)
    mask[lat_index, lon_index] = 1
    mask = mask.astype(bool)

    end = time.time()
    print(f'DONE masking raster data with arbitrary shape in {end - start} seconds')
    print('### ~~~~~~ ###')

    # visualize the shape, all pixels and masked pixels
    m = gdf_shape.explore(name='shape', tiles='Stamen Terrain')
    gdf_2d.explore(m=m, column='t2m', name='api_request_points', cmap='Oranges')
    gdf_masked.explore(m=m, column='t2m', name='masked_points', cmap='Blues')
    folium.LayerControl().add_to(m)
    # m.save('poc_a_map.html')
    return mask, m

# Working example

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('data/vector/greenland_main_island.geojson')  # read the shape from a geojson file
shape = gdf.loc[0, 'geometry']  # get an object of an arbitrary shape

# request ERA5 API for whole month
day = [str(i).zfill(2) for i in range(1, 32)]
fpath = request_era5_api('poc_e1.nc', shape=shape, day=day)
mask, m = shape_mask(fpath, shape)